In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [15]:
import osmnx as ox
import pandas as pd
import numpy as np
from Data import Data 
import networkx as nx
from networkx import algorithms 
import math
from numpy import arctan2, sin, cos, sqrt, radians

In [3]:
graph = ox.graph_from_address('Athens, Municipality of Athens, Regional Unit of Central Athens, Attica, 10667, Greece', network_type='drive') 

In [10]:
df = pd.read_pickle("Data_sample.pkl")
X = df.copy()

In [5]:
#get the nearest edge and node for each location:
nodes = np.zeros(len(df))
EdgeStart = np.zeros(len(df))
EdgeEnd = np.zeros(len(df))
for i in range(len(df)):
    tup1 = (df.iloc[i]['lat'], df.iloc[i]['lon'])
    nodes[i] = ox.get_nearest_node(graph, tup1 , method='euclidean')
    start, end, c = ox.get_nearest_edge(graph, tup1)
    EdgeStart[i] = start
    EdgeEnd[i] = end

In [17]:
def __construct_graph_data_cols(graph):
    def aux(row):
        coord = (row['lat'],row['lon'])
        nn = ox.get_nearest_node(graph, coord, method='euclidean')
        start, end, _ = ox.get_nearest_edge(graph, coord)
        edge_prog = __edge_progress(graph, start, end, coord)
        start_lon = graph.nodes[start]['x']
        start_lat = graph.nodes[start]['y']
        end_lon = graph.nodes[end]['x']
        end_lat = graph.nodes[end]['y']
        return nn, start, start_lon, start_lat, end, end_lon, end_lat, edge_prog
    return aux

In [18]:
 df['nearest_node'],
    df['nearest_edge_start_node'],
    df['nearest_edge_start_node_lon'],
    df['nearest_edge_start_node_lat'],
    df['nearest_edge_end_node'],
    df['nearest_edge_end_node_lon'],
    df['nearest_edge_end_node_lat'],
    df['edge_progress'] =zip(*df.apply(__construct_graph_data_cols(graph), axis=1))

In [19]:
df

lat        lon    speed  lon_acc  lat_acc  type  traveled_d  \
id time                                                                       
1  0.00   37.977482  23.735405  34.0206  -0.0167   0.0074  Taxi      320.26   
   0.04   37.977485  23.735402  34.0197   0.0040   0.0086  Taxi      320.26   
   0.08   37.977488  23.735400  34.0211   0.0167   0.0097  Taxi      320.26   
   0.12   37.977491  23.735398  34.0245   0.0294   0.0108  Taxi      320.26   
   0.16   37.977494  23.735395  34.0305   0.0540   0.0120  Taxi      320.26   
...             ...        ...      ...      ...      ...   ...         ...   
5  29.24  37.979836  23.733313  36.8072  -3.7893   1.1652   Car      272.25   
   29.28  37.979838  23.733311  36.4768  -3.8277   1.1690   Car      272.25   
   29.32  37.979841  23.733309  36.1375  -3.8118   1.1665   Car      272.25   
   29.36  37.979844  23.733306  35.7859  -3.8818   1.1568   Car      272.25   
   29.40  37.979847  23.733304  35.4092  -4.0068   1.1387   Car      272.25   

          avg_speed  nearest_node  nearest_edge_start_node  \
id time                                                      
1  0.00   35.366264    3339821648               3339821648   
   0.04   35.366264    3339821648               3339821648   
   0.08   35.366264    3339821648               3339821648   
   0.12   35.366264    3339821648               3339821648   
   0.16   35.366264    3339821648               3339821648   
...             ...           ...                      ...   
5  29.24  33.336716     250698925                250698924   
   29.28  33.336716     250698925                250698924   
   29.32  33.336716     250698925                250698924   
   29.36  33.336716     250698925                250698924   
   29.40  33.336716     250698925                250698924   

          nearest_edge_start_node_lon  nearest_edge_start_node_lat  \
id time                                                              
1  0.00                     23.735888                    37.976884   
   0.04                     23.735888                    37.976884   
   0.08                     23.735888                    37.976884   
   0.12                     23.735888                    37.976884   
   0.16                     23.735888                    37.976884   
...                               ...                          ...   
5  29.24                    23.733820                    37.979335   
   29.28                    23.733820                    37.979335   
   29.32                    23.733820                    37.979335   
   29.36                    23.733820                    37.979335   
   29.40                    23.733820                    37.979335   

          nearest_edge_end_node  nearest_edge_end_node_lon  \
id time                                                      
1  0.00               250691723                  23.734863   
   0.04               250691723                  23.734863   
   0.08               250691723                  23.734863   
   0.12               250691723                  23.734863   
   0.16               250691723                  23.734863   
...                         ...                        ...   
5  29.24              250698925                  23.732912   
   29.28              250698925                  23.732912   
   29.32              250698925                  23.732912   
   29.36              250698925                  23.732912   
   29.40              250698925                  23.732912   

          nearest_edge_end_node_lat  edge_progress  
id time                                             
1  0.00                   37.978080       0.490988  
   0.04                   37.978080       0.493622  
   0.08                   37.978080       0.495962  
   0.12                   37.978080       0.498301  
   0.16                   37.978080       0.500935  
...                             ...            ...  
5  29.24                  37.980373    

In [4]:
#my first draft of the function 
def Distance_achieved(edge_node1,edge_node2,lonV, latV):
    #assuming that edge_node1 is the starting point
    lon0 = graph.nodes[edge_node1]['x']
    lon1 = graph.nodes[edge_node2]['x']
    lat0 = graph.nodes[edge_node1]['y']
    lat1 = graph.nodes[edge_node2]['y']
    a = Distance_Euc(lon0, lat0, lon1, lat1)
    b = Distance_Euc(lon0, lat0, lonV, latV)
    return b/a

In [5]:
#my first draft of the function 
def Distance_Euc(lon0, lat0, lon1, lat1):
    lon0 = math.radians(lon0)
    lat0 = math.radians(lat0)
    lon1 = math.radians(lon1)
    lat1 = math.radians(lat1)
    dlon = lon1 - lon0
    dlat = lat1 - lat0
    a = math.sin(dlat / 2)**2 + math.cos(lat0) * math.cos(lat1) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return 6373.0*c

In [12]:
def __edge_progress(graph, edge_start_node, edge_end_node, v_coord):
    start_coord = graph.nodes[edge_start_node]['y'], graph.nodes[edge_start_node]['x']
    end_coord = graph.nodes[edge_end_node]['y'], graph.nodes[edge_end_node]['x']

    a = __euc_dist(start_coord, end_coord)
    b = __euc_dist(start_coord, v_coord)
    return b/a


def __euc_dist(coord0, coord1):
    EARTH_RADIUS = 6373

    lat0, lon0 = coord0
    lat1, lon1 = coord1

    lat0, lon0 = radians(lat0), radians(lon0)
    lat1, lon1 = radians(lat1), radians(lon1)

    dlat = lat1 - lat0
    dlon = lon1 - lon0

    a = sin(dlat/2)**2 + cos(lat0) * cos(lat1) * sin(dlon/2)**2
    c = 2 * arctan2(sqrt(a), sqrt(1-a))
    return EARTH_RADIUS*c
